In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.2.0'

In [5]:
import tensorflow.keras.layers as L
from tensorflow.keras import Model

In [6]:
mnist = tf.keras.datasets.mnist

In [20]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = tf.image.convert_image_dtype(x_train, tf.float32)[..., tf.newaxis]
x_test = tf.image.convert_image_dtype(x_test, tf.float32)[..., tf.newaxis]

In [22]:
x_train.shape

TensorShape([60000, 28, 28, 1])

In [24]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [29]:
class MiModel(Model):
    def __init__(self):
        super(MiModel, self).__init__()
        self.conv1 = L.Conv2D(32, 3, activation="relu")
        self.flatten = L.Flatten()
        self.d1 = L.Dense(100, activation="relu")
        self.d2 = L.Dense(10, activation="softmax")

    def call(self, x):
        x = self.conv1(x)
        x = self.d1(self.flatten(x))
        x = self.d2(x)
        return x

In [30]:
model = MiModel()

In [32]:
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name="train_loss")
train_acc = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

test_loss = tf.keras.metrics.Mean(name="test_loss")
test_acc = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")

In [33]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        preds = model(images, training=True)
        loss = loss_obj(labels, preds)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_acc(labels, preds)

In [38]:
@tf.function
def test_step(images, labels):
    preds = model(images, training=False)
    loss = loss_obj(labels, preds)
    
    test_loss(loss)
    test_acc(labels, preds)

In [40]:
EPOCHS = 5

for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_acc.reset_states()
    test_loss.reset_states()
    test_acc.reset_states()
    
    for images, labels in train_dataset:
        train_step(images, labels)
        
    for images, labels in test_dataset:
        test_step(images, labels)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                        train_loss.result(),
                        train_acc.result() * 100,
                        test_loss.result(),
                        test_acc.result() * 100))

Epoch 1, Loss: 1.4794639348983765, Accuracy: 98.26166534423828, Test Loss: 1.48145592212677, Test Accuracy: 98.0199966430664
Epoch 2, Loss: 1.4751551151275635, Accuracy: 98.67500305175781, Test Loss: 1.4815365076065063, Test Accuracy: 97.97000122070312
